In [1]:
from twenty_forty_eight_linux import TwentyFortyEight
import numpy as np
#from scipy.stats import rv_discrete
from collections import deque
import pickle
import logging

In [2]:
def discounted_reward(rewards, gamma=0.95):
    r = np.asarray(rewards)
    g = np.full_like(rewards, gamma, dtype=np.float64) ** np.arange(len(rewards))
    return np.sum(r * g)

In [3]:
def epsilon_greedy_policy(q_vector, epsilon):
    prob_max = epsilon / 4 + 1 - epsilon
    prob_random = epsilon / 4
    is_greedy_action = False if np.random.uniform() <= epsilon else True
    if is_greedy_action:
        #Random choice, if there are at least 2 Q-s with the same values
        # TODO: Check, if randomizing the q_vector in lookup_action_value() around 0, what would happen
        max_q_args = np.argwhere(q_vector == np.amax(q_vector))
        if len(max_q_args) > 1:
            action = np.random.choice(max_q_args.ravel(), 1)[0] + 1
        else:
            action = np.argmax(q_vector) + 1
    else:
        action = np.random.randint(1, 5)
    return action

In [4]:
def action_value_update(q, discounted_reward, n):
    return q + (discounted_reward - q) / n

In [5]:
def lookup_action_value(state):
    if state not in action_values_table:
        action_values_table[state] = [0, 0, 0, 0]
    return action_values_table[state]

In [6]:
def lookup_action_value_wo_update(state):
    av = action_values_table.get(state)
    return av if av else [0, 0, 0, 0]

In [ ]:
action_values_table = {}
#with open("c:\\Work\\Jupyter\\2048\\action_value_dict.p", "rb") as f:
#    action_values_table = pickle.load(f)

epsilon = 1
episode_number = 0

#Külső ciklus
for _ in range(8000):
    print("-", end="")
    episode_number += 1
    if episode_number % 500 == 0:
        print(str(episode_number) + ". játék")
        print("Lépések száma: " + str(len(actions_deque)))
        print("Maximum kocka: " + str(max(game_state)))
        
        with open("c:\\Work\\Jupyter\\2048\\action_value_dict_-10_reward.p", "wb") as f:
            pickle.dump(action_values_table, f)

    epsilon = 1 / episode_number
    # Containers
    states_input_deque = deque()
    states_reward_list = []
    actions_deque = deque()
    # End bool
    is_ended = False
    # New game
    game = TwentyFortyEight(4, 4)
    game_state = game.table_as_array()
    #Belső ciklus
    while not is_ended:
        # Append current game state
        states_input_deque.append(game_state)

        # Lookup action value belonging to current state
        action_value_vector = lookup_action_value(game_state)

        # Choose action based on current action value vector and epsilon
        action = epsilon_greedy_policy(action_value_vector, epsilon)
        # Append action
        actions_deque.append(action)

        # Make the move in game (environment)
        game.move(action)
        # Get next state, reward
        game_state, reward, is_ended = game.table_as_array(), game.reward(), game.is_ended()

        # Append reward
        states_reward_list.append(reward)

    # Update action values loop
    for i, (state, action) in enumerate(zip(states_input_deque, actions_deque)):
        action_value_vect = lookup_action_value(state)
        disc_rew = discounted_reward(states_reward_list[i:])
        q = action_values_table[state][action - 1]
        action_values_table[state][action - 1] = action_value_update(q, disc_rew, len(states_reward_list[i:]))


In [11]:
ll = deque()
for k in action_values_table.keys():
    ll.append(max(k))
max(ll)

512

In [62]:
for i, k in enumerate(action_values_table.values()):
    if 1000 < i < 1015:
        print(k)
    if i == 1015:
        break
    

[0, 0.049557621876696947, 0, 0]
[0, 0, 0, 0.011706483942031597]
[0, -0.030793333788485346, 0, -0.056989276610516483]
[0, 0, 0, -0.012301362843213282]
[0.019017344926190935, 0, 0, 0]
[0, 0.046853204668625081, 0, 0]
[0, 0, 0.016144215584519096, 0]
[0.0062272031916750063, 0.013388412209856034, 0, 0]
[0, 0, 0.05384597928481278, 0]
[-0.0065776768127108068, 0, 0, 0]
[0, 0, 0.032209340821758276, 0]
[0, 0, 0, 0.025075756475254174]
[0, 0, -0.23508412184258323, 0]
[0.05364018425417711, 0, 0, 0]


deque([])

In [8]:
def try_table():
    gam = TwentyFortyEight(4, 4)
    game_state = gam.table_as_array()
    is_ended = False
    states_input_deque = deque()
    states_reward_list = []
    actions_deque = deque()
    
    while not is_ended:
        # Append current game state
        states_input_deque.append(game_state)

        # Lookup action value belonging to current state
        action_value_vector = lookup_action_value_wo_update(game_state)

        # Choose action based on current action value vector and epsilon
        action = epsilon_greedy_policy(action_value_vector, epsilon)
        # Append action
        actions_deque.append(action)

        # Make the move in game (environment)
        gam.move(action)
        # Get next state, reward
        game_state, reward, is_ended = gam.table_as_array(), gam.reward(), gam.is_ended()

        # Append reward
        states_reward_list.append(reward)
    
    return states_input_deque, states_reward_list, actions_deque, gam

In [94]:
with open("/home/atoth/Jupyter_notebooks/2048/action_value_dict_2016_10_08_19_10.p", "rb") as f:
    action_values_table = pickle.load(f)

In [92]:
np.sum(np.asarray([sum(try_table()[0][-1]) for ga in range(3000)])) / 3000

263.91866666666664

In [106]:
aaa = []
for v in action_values_table.values():
    if v.count(0) == 0:
        aaa.append(v) 
len(aaa)

130

In [87]:
np.sum(np.asarray([sum(try_table()[0][-1]) for ga in range(1000)])) / 1000

260.60700000000003

In [52]:
len(action_values_table)

924478

In [25]:
a, b, c, gam = try_table()

In [26]:
print(gam)

2 ,4  ,8 ,2 
4 ,2  ,64,4 
2 ,128,2 ,16
16,4  ,16,4 
